In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import numpy as np
from math import radians, cos, sin, asin, sqrt, hypot
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from catboost import *
#%matplotlib notebook
from category_encoders import *
np.random.seed(0)

# Решение

### Что подошло.

1. В изначальном датасете к координатам, данным в условии, подкручивались города России с двумя признаками: население и дистанция от точки до центра города. Города взяты по [ссылке](https://gist.github.com/nalgeon/5307af065ff0e3bc97927c832fabe26b). Встречались точки не из России, к ним сопоставлялся ближайший Российский город (с расстоянием и населением). Эти агрегированные данные сохранялись в новом файле train_merged.csv и test_merged.csv, которые можно выкачать [отсюда. ](https://drive.google.com/file/d/1f1K9iw0riUU2v_nGSfks_0TOrm_n7Uru/view?usp=sharing)
2. В изначальных данных оказались столбцы, которых не было в тестовых (cancel_time, driver_found). Из этих данных я выделил потенциально мертвые точки, то есть те точки, для которых отмена заказа произошла спустя 1200 секунд и более и водитель так и не был найден. Затем для каждой точки из датасетов я посчитал расстояние до ближайшей мертвой точки и включил это расстояние как признак `is_bad`. Этот признак принес мне около 1% точности.
3. Выбросил выбросы по широте/долготе.
4. Последнее, что я выяснил, что признак "час дня" очень хорошо отражал сгораемость заказа. Видимо, люди более нетерпеливы по утрам, чем днём, что достаточно логично. Этот признак принёс около 1-2% точности.
5. Все категориальные признаки я кодировал с помощью библиотеки `category_encoders`, методом `JamesSteinEncoder`. Другие методы кодировки приносили меньшую точность. Отдельно скажу, что NaN's в классах автомобиля я кодировал отдельным классом, потому что пропусков было много.
6. Обучал бустинг на сбалансированных данных. Данные балансировал в соотношении 2 к 1. (2 не сгоревших заказа на 1 сгоревший).
7. Модель обучал на гпу, деревья строились нестандартно (в глубину, не параллельно), и что самое интересное и непонятное для меня - качество модели на отложенной выборке не совпдало с качеством на кагле.
8. Параметры бустинга подбирал поиском по сетке, хотя опять же, непонятно было, что считать критерием хорошего качетсва подобранных параметров, если не всегда качество на отложенной выборке совпадало с качеством на кагле.

### Что не подошло.

1. Чем больше признаков я добавлял, тем более зашумленными становились данные, поэтому пришлось отказаться от признаков население города и расстояние до центра города. 
2. Пришлось отказаться от признаков `weekday`, `day`, в силу того, что они также не добавляли к качеству.
3. На качество также негативно влияли признаки стоимость нефти, стоимость доллара. Исключаем.
4. Пробовал бустинг на глубине 10 в количестве 1000 и более деревьев. Несмотря на то, что на отложенной выборке рост AUC-ROC продолжался, на кагле наблюдалось явное переобучение.

    
## Итоги.

1. Точность на Public Leaderboard: 0.61320
2. Точность на Private Leaderboard: 0.61638

**Признаки которые я использовал**:

    'f_class'
    'lat'
    'lon'
    's_class'
    't_class'
    'is_bad'
    'hour'



### Подгрузка данных

Класс `DataLoader` загружает данные и проводит первичную обработку, а именно: 
1. Выделяет признаки `day`, `weekday`, `hour`.
2. Дропает неподходящие признаки, а именно: `due`, `Население`, `Город`, `Федеральный округ`. 
3. Считает расстояние до ближайших точек, время ожидания такси в которых превысило параметр `time_wait`.

In [2]:
# Время исполнения ячейки ~ 10-15 минут.

class DataLoader():
    def __init__(self, time_wait=None):
        '''
        time_wait: добавляет признак, сколько ждал клиент, пока не отменил заказ и водитель не приехал...
        ...чтобы выявить потенциально плохие места заказа такси.
        '''
        self.time_wait = time_wait
        self.train, self.test = self.prepare_train(pd.read_csv('train_merged.csv', index_col=0), time_wait), self.prepare_test(pd.read_csv('test_merged.csv', index_col=0))
        self.bad_points = None
        
    def prepare_train(self, df, dist_to_bad=None):
        df['hour'] = pd.DatetimeIndex(pd.to_datetime(df['due'])).hour
        df['day'] = pd.DatetimeIndex(pd.to_datetime(df['due'])).day
        df['weekday'] = pd.DatetimeIndex(pd.to_datetime(df['due'])).weekday
        df = df.drop(['due', 'Население', 'Город', 'Федеральный округ'], axis=1)
        if dist_to_bad is None:
            return df
        print('Подготовка трейн датасета')
        self._get_bad_points(df, dist_to_bad)
        df = self._get_dist_to_bad_points(df)
        self.time_wait = 'trained'
        return df
    
    def prepare_test(self, df):
        df['hour'] = pd.DatetimeIndex(pd.to_datetime(df['due'])).hour
        df['day'] = pd.DatetimeIndex(pd.to_datetime(df['due'])).day
        df['weekday'] = pd.DatetimeIndex(pd.to_datetime(df['due'])).weekday
        df = df.drop(['due', 'Население', 'Город', 'Федеральный округ'], axis=1)
        if self.bad_points is not None:
            print('Подготовка тест датасета')
            df = self._get_dist_to_bad_points(df)
        return df
            
            
    def _get_bad_points(self, df, time_waited):
        self.bad_points = df[(df['cancel_time'] > time_waited) & (df['driver_found'] == 0)][['lat', 'lon']]
        return self.bad_points
    
    def _get_dist_to_bad_points(self, df):
        bad_points = self.bad_points.values
        
        def haversine(a):
            # Фнукцию взял из интернета, но переделал под векторное использование, чтобы ускорить работу алгоритма
            a, bad_points_ = np.radians(a.values), np.radians(self.bad_points.values)
            dlon = bad_points_[:, 1] - a[1]
            dlat = bad_points_[:, 0] - a[0]
            temp = np.power(np.sin(dlat / 2), 2) + np.cos(a[0])*np.cos(bad_points_[:, 0]) * np.power(np.sin(dlon / 2), 2)
            c = 2 * np.arcsin(np.sqrt(temp))
            km = 6371 * c
            return km.min()
        
        tqdm.tqdm().pandas()
        # Связка плохих точек и всех остальных
        df['is_bad'] = df[['lat', 'lon']].progress_apply(haversine, axis=1)
        return df
    
data = DataLoader(1200)

# data.train возвращает датасет для обучения
# data.test возвращает датасет для валидации

d:\python\venvs\new_project\env\lib\site-packages\numpy\lib\arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
0it [00:00, ?it/s]
d:\python\venvs\new_project\env\lib\site-packages\tqdm\std.py:656: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
  0%|                                                                          | 301/1450000 [00:00<08:06, 2981.39it/s]

Подготовка трейн датасета


100%|██████████████████████████████████████████████████████████████████████| 1450000/1450000 [05:55<00:00, 4076.94it/s]
0it [00:00, ?it/s]
  0%|                                                                           | 342/343300 [00:00<01:40, 3417.36it/s]

Подготовка тест датасета


100%|████████████████████████████████████████████████████████████████████████| 343300/343300 [01:25<00:00, 3995.39it/s]


### Кодирование категориальных признаков

Класс `Coder` помогал мне перебирать различные способы кодировки, чтобы тестировать влияние кодировки на целевую переменную.

In [3]:
class Coder():
    def __init__(self, train, test, labels, cols):
        self.train, self.test, self.labels, self.cols = train, test, labels, cols
        
    def base(self):
        cod = BaseNEncoder(cols=self.cols).fit(self.train, self.labels)
        return cod.transform(self.train), cod.transform(self.test)
    
    def target_enc(self):
        cod = TargetEncoder(cols=self.cols).fit(self.train, self.labels)
        return cod.transform(self.train), cod.transform(self.test)
    
    def backward(self):
        cod = BackwardDifferenceEncoder(cols=self.cols).fit(self.train, self.labels)
        return cod.transform(self.train), cod.transform(self.test)
    
    def james_stein(self):
        cod = JamesSteinEncoder(cols=self.cols).fit(self.train, self.labels)
        return cod.transform(self.train), cod.transform(self.test)
    
    def m_estimate(self):
        cod = MEstimateEncoder(cols=self.cols).fit(self.train, self.labels)
        return cod.transform(self.train), cod.transform(self.test)

In [6]:
train = data.train
test = data.test

# Выкидываем аутлаеры по долготе и широте 
train = train.drop(train[(train['cancel_time'] > 15000) & (train['driver_found'] == 1)].index)
train_data = train[(train['lat'] > 51) & (train['lon'] < 49) & (train['lon'] > 30)]
train_data = train_data[(train_data['cancel_time'] < 20000)]


labels = train_data['burned']
del train_data['burned']
train_data = train_data.drop(['cancel_time', 'driver_found'], axis=1)

# Разные столбцы кодируем по-разному
cols = ['f_class', 's_class', 't_class']
coder = Coder(train_data, test, labels, cols)
train_data, test_data = coder.james_stein()

# train_data = train_data.drop(['hour', 'day', 'weekday'], axis=1)
# test_data = test_data.drop(['hour', 'day', 'weekday'], axis=1)
cols = ['hour', 'day', 'weekday']
coder = Coder(train_data, test_data, labels, cols)
train_data, test_data = coder.james_stein()

# Добавляем лэйблы в датасет
train_data = train_data.join(labels)

# Балансируем выборку
burned = train_data[train_data['burned'] == 1]
train_data = train_data[train_data['burned'] == 0].sample(2 * len(burned), random_state=58).merge(burned, how='outer')
train_data = train_data.sample(len(train_data), random_state=58)
labels = train_data['burned']
del train_data['burned']
# del train_data['date']
# del test_data['date']

In [7]:
# Выбираем только те признаки, которые нам больше всего подходят
train_data_ = train_data[['f_class', 's_class', 't_class', 'lat', 'lon', 'hour', 'is_bad']]
test_data_ = test_data[['f_class', 's_class', 't_class','lat', 'lon', 'hour', 'is_bad']]

In [8]:
# Генерируем отложенную выборку
train_, test_, train_labels, test_labels = train_test_split(train_data_, labels, shuffle=True, random_state=69)

In [9]:
# Запускаем процесс обучения
boost = CatBoostClassifier(iterations=195,
                           depth=9,
                           task_type='GPU',
                           verbose=True,
                           eval_metric='AUC',
                           use_best_model=True,
                           one_hot_max_size=250,
                           learning_rate=0.07,
                           grow_policy='Depthwise',
                           border_count=255,
                           random_seed=5,
                           early_stopping_rounds=10
                          )

eval_dataset = Pool(test_,
                    test_labels)

boost.fit(train_, train_labels,  eval_set=eval_dataset)

0:	learn: 0.5860070	test: 0.5827580	best: 0.5827580 (0)	total: 94.8ms	remaining: 18.4s
1:	learn: 0.5912350	test: 0.5874505	best: 0.5874505 (1)	total: 196ms	remaining: 18.9s
2:	learn: 0.5921421	test: 0.5885005	best: 0.5885005 (2)	total: 289ms	remaining: 18.5s
3:	learn: 0.5933520	test: 0.5894898	best: 0.5894898 (3)	total: 386ms	remaining: 18.4s
4:	learn: 0.5946159	test: 0.5905711	best: 0.5905711 (4)	total: 475ms	remaining: 18.1s
5:	learn: 0.5957351	test: 0.5915063	best: 0.5915063 (5)	total: 560ms	remaining: 17.6s
6:	learn: 0.5963869	test: 0.5918407	best: 0.5918407 (6)	total: 655ms	remaining: 17.6s
7:	learn: 0.5968417	test: 0.5920203	best: 0.5920203 (7)	total: 748ms	remaining: 17.5s
8:	learn: 0.5970187	test: 0.5921411	best: 0.5921411 (8)	total: 841ms	remaining: 17.4s
9:	learn: 0.5977485	test: 0.5924622	best: 0.5924622 (9)	total: 937ms	remaining: 17.3s
10:	learn: 0.5982856	test: 0.5928413	best: 0.5928413 (10)	total: 1.04s	remaining: 17.5s
11:	learn: 0.5988991	test: 0.5932311	best: 0.593231

Как видно из логов обучения AUC-ROC на отложенной выборке составлял 0.626, что соответствует качеству в 0.612 на Public Leaderboard.

### Отправка предсказаний

In [758]:
probas = boost.predict_proba(test_data)[:, 1]
answers = pd.DataFrame(probas, columns=['Prob'])
answers.to_csv('new.csv', index_label='Id')

### Grid Search

In [235]:
from catboost import *
grid = {'learning_rate': [0.03, 0.06, 0.1],
        'depth': [6, 7, 8, 9, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}
id = 0
data = {}
for lr in grid['learning_rate']:
    for dept in grid['depth']:
        for leaf in grid['l2_leaf_reg']:
            id += 1
            boost = CatBoostClassifier(iterations=10,
                                       depth=dept,
                                       task_type='GPU',
                                       eval_metric='AUC',
                                       use_best_model=True,
                                       one_hot_max_size=250,
                                       learning_rate=lr,
                                       verbose=500,
                                       l2_leaf_reg=leaf
                                      )

            eval_dataset = Pool(test_,
                                test_labels)#, #cat_features=cat)

            # train the model
            boost.fit(train_, train_labels,  eval_set=eval_dataset)

            data.update({id: {'test_auc': boost.get_best_score()['validation']['AUC'],
                              'train_auc': boost.get_best_score()['learn']['AUC'],
                              'learning_rate': lr,
                              'l2_leaf_reg': leaf,
                              'depth': dept
                             }})

0:	learn: 0.5559839	test: 0.5562822	best: 0.5562822 (0)	total: 72.9ms	remaining: 1m 12s
500:	learn: 0.5910378	test: 0.5893124	best: 0.5893124 (500)	total: 39.2s	remaining: 39s
999:	learn: 0.5987374	test: 0.5939363	best: 0.5939363 (999)	total: 1m 19s	remaining: 0us
bestTest = 0.5939362645
bestIteration = 999
0:	learn: 0.5559839	test: 0.5562822	best: 0.5562822 (0)	total: 76.8ms	remaining: 1m 16s
500:	learn: 0.5910043	test: 0.5894117	best: 0.5894117 (500)	total: 40.3s	remaining: 40.1s
999:	learn: 0.5985901	test: 0.5938542	best: 0.5938542 (999)	total: 1m 22s	remaining: 0us
bestTest = 0.5938542485
bestIteration = 999
0:	learn: 0.5559839	test: 0.5562822	best: 0.5562822 (0)	total: 83.9ms	remaining: 1m 23s
500:	learn: 0.5911259	test: 0.5894412	best: 0.5894412 (500)	total: 43.4s	remaining: 43.2s
999:	learn: 0.5985673	test: 0.5939462	best: 0.5939462 (999)	total: 1m 23s	remaining: 0us
bestTest = 0.5939461887
bestIteration = 999
0:	learn: 0.5559839	test: 0.5562822	best: 0.5562822 (0)	total: 76.2ms